# 星际争霸-层级索引学习

In [1]:
import pandas as pd

dataset_path = './starcraft.csv'
df_data = pd.read_csv(dataset_path, usecols=['LeagueIndex', 'Age', 'HoursPerWeek',
                                             'TotalHours', 'APM'])
df_data

,LeagueIndex,Age,HoursPerWeek,TotalHours,APM
0,5,27.0,10.0,3000.0,143.7180
1,5,23.0,10.0,5000.0,129.2322
2,4,30.0,10.0,200.0,69.9612
3,3,19.0,20.0,400.0,107.6016
4,3,32.0,10.0,500.0,122.8908
...,...,...,...,...,...
3390,8,NaN,NaN,NaN,259.6296
3391,8,NaN,NaN,NaN,314.6700
3392,8,NaN,NaN,NaN,299.4282
3393,8,NaN,NaN,NaN,375.8664


In [2]:
def top_n(df, n=3, column='APM'):
    """
        返回每个分组按 column 的 top n 数据
    """
    return df.sort_values(by=column, ascending=False)[:n]

print(df_data.groupby('LeagueIndex').apply(top_n))

                  LeagueIndex   Age  HoursPerWeek  TotalHours       APM
LeagueIndex                                                            
1           2214            1  20.0          12.0       730.0  172.9530
            2246            1  27.0           8.0       250.0  141.6282
            1753            1  20.0          28.0       100.0  139.6362
2           3062            2  20.0           6.0       100.0  179.6250
            3229            2  16.0          24.0       110.0  156.7380
            1520            2  29.0           6.0       250.0  151.6470
3           1557            3  22.0           6.0       200.0  226.6554
            484             3  19.0          42.0       450.0  220.0692
            2883            3  16.0           8.0       800.0  208.9500
4           2688            4  26.0          24.0       990.0  249.0210
            1759            4  16.0           6.0        75.0  229.9122
            2637            4  23.0          24.0       650.0  2

In [3]:
# apply函数接收的参数会传入自定义的函数中；分别应用于groupby后的每一组
m = df_data.groupby('LeagueIndex').apply(top_n, n=2, column='Age')
m.index

MultiIndex([(1, 3146),
            (1, 3040),
            (2,  920),
            (2, 2437),
            (3, 1258),
            (3, 2972),
            (4, 1696),
            (4, 1729),
            (5,  202),
            (5, 2745),
            (6, 3069),
            (6, 2706),
            (7, 2813),
            (7, 1992),
            (8, 3340),
            (8, 3341)],
           names=['LeagueIndex', None])

In [4]:
# group_keys为False后，被groupby的列不会变为索引
n=df_data.groupby('LeagueIndex', group_keys=False).apply(top_n)
# n
n.index

Int64Index([2214, 2246, 1753, 3062, 3229, 1520, 1557,  484, 2883, 2688, 1759,
            2637, 3277,   93,  202,  734, 2746, 1810, 3127,  104, 1654, 3393,
            3373, 3372],
           dtype='int64')

# 10 数据清洗（重复数据处理 与 连接）

In [5]:
import pandas as pd
import numpy as np

df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                        'data1' : np.random.randint(0,10,7)})
df_obj2 = pd.DataFrame({'key': ['a', 'b' ,'d'],
                        'data2' : np.random.randint(0,10,3)})

print(df_obj1)
print('-'*50)
print(df_obj2)

  key  data1
0   b      6
1   b      2
2   a      1
3   c      7
4   a      6
5   a      8
6   b      1
--------------------------------------------------
  key  data2
0   a      9
1   b      2
2   d      4


In [6]:
#默认连接使用相同的列名，连接方式是内连接
pd.merge(df_obj1, df_obj2)

,key,data1,data2
0,b,6,2
1,b,2,2
2,b,1,2
3,a,1,9
4,a,6,9
5,a,8,9


In [8]:
#左表和右表都拿索引连接
pd.merge(df_obj1, df_obj2, left_index=True, right_index=True)

,key_x,data1,key_y,data2
0,b,6,a,9
1,b,2,b,2
2,a,1,d,4


In [10]:
# 更改列名
df_obj1 = df_obj1.rename(columns={'key':'key1'})
df_obj2 = df_obj2.rename(columns={'key':'key2'})
print(df_obj1)
df_obj2

  key1  data1
0    b      6
1    b      2
2    a      1
3    c      7
4    a      6
5    a      8
6    b      1


,key2,data2
0,a,9
1,b,2
2,d,4


In [11]:
#左表以key1来连接，右表以key2来连接
pd.merge(df_obj1, df_obj2, left_on="key1", right_on="key2")

,key1,data1,key2,data2
0,b,6,b,2
1,b,2,b,2
2,b,1,b,2
3,a,1,a,9
4,a,6,a,9
5,a,8,a,9


In [12]:
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='outer')  #全外连接

,key1,data1,key2,data2
0,b,6.0,b,2.0
1,b,2.0,b,2.0
2,b,1.0,b,2.0
3,a,1.0,a,9.0
4,a,6.0,a,9.0
5,a,8.0,a,9.0
6,c,7.0,NaN,NaN
7,NaN,NaN,d,4.0


In [13]:
#left 等价于 left join 等价于 left outer join
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='left')  # 只比内连接多保留c

,key1,data1,key2,data2
0,b,6,b,2.0
1,b,2,b,2.0
2,a,1,a,9.0
3,c,7,NaN,NaN
4,a,6,a,9.0
5,a,8,a,9.0
6,b,1,b,2.0


In [14]:
# right等价于数据库的right join 等价于right outer join
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='right')  # 只比内连接多保留d

,key1,data1,key2,data2
0,a,1.0,a,9
1,a,6.0,a,9
2,a,8.0,a,9
3,b,6.0,b,2
4,b,2.0,b,2
5,b,1.0,b,2
6,NaN,NaN,d,4


In [18]:
# 处理重复列名
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                        'data' : np.random.randint(0,10,7)})
df_obj2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                        'data' : np.random.randint(0,10,3)})
#给相同的数据列添加后缀
print(pd.merge(df_obj1, df_obj2, on='key', suffixes=('_left', '_right')))

  key  data_left  data_right
0   b          4           3
1   b          5           3
2   b          8           3
3   a          9           3
4   a          2           3
5   a          8           3


In [21]:
# 按索引连接
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                        'data1' : np.random.randint(0,10,7)})
df_obj2 = pd.DataFrame({'data2' : np.random.randint(0,10,3)}, index=['a', 'b', 'd'])
print(df_obj1)
print(df_obj2)
print(pd.merge(df_obj1, df_obj2, left_on='key', right_index=True))

  key  data1
0   b      9
1   b      2
2   a      9
3   c      7
4   a      3
5   a      9
6   b      4
   data2
a      4
b      6
d      3
  key  data1  data2
0   b      9      6
1   b      2      6
6   b      4      6
2   a      9      4
4   a      3      4
5   a      9      4


In [22]:
pd.merge(df_obj2,df_obj1, left_index=True, right_on='key')

,data2,key,data1
2,4,a,9
4,4,a,3
5,4,a,9
0,6,b,9
1,6,b,2
6,6,b,4


# 11 数据合并(pd.concat)

In [23]:
# np里边的合并
arr1 = np.random.randint(0, 10, (3, 4))
arr2 = np.random.randint(0, 10, (3, 4))

print(arr1)
print(arr2)

print(np.concatenate([arr1, arr2]))
print(np.concatenate([arr1, arr2], axis=1))

[[9 9 2 9]
 [2 6 3 1]
 [2 5 5 6]]
[[4 6 1 0]
 [9 8 8 1]
 [6 0 3 8]]
[[9 9 2 9]
 [2 6 3 1]
 [2 5 5 6]
 [4 6 1 0]
 [9 8 8 1]
 [6 0 3 8]]
[[9 9 2 9 4 6 1 0]
 [2 6 3 1 9 8 8 1]
 [2 5 5 6 6 0 3 8]]


In [24]:
# index 没有重复的情况
ser_obj1 = pd.Series(np.random.randint(0, 10, 5), index=range(0,5))
ser_obj2 = pd.Series(np.random.randint(0, 10, 4), index=range(5,9))
ser_obj3 = pd.Series(np.random.randint(0, 10, 3), index=range(9,12))

print(ser_obj1)
print(ser_obj2)
print(ser_obj3)

0    3
1    6
2    1
3    7
4    1
dtype: int32
5    0
6    1
7    7
8    3
dtype: int32
9     8
10    2
11    8
dtype: int32


In [25]:
print(pd.concat([ser_obj1, ser_obj2, ser_obj3]))
print(pd.concat([ser_obj1, ser_obj2, ser_obj3], axis=1))

0     3
1     6
2     1
3     7
4     1
5     0
6     1
7     7
8     3
9     8
10    2
11    8
dtype: int32
      0    1    2
0   3.0  NaN  NaN
1   6.0  NaN  NaN
2   1.0  NaN  NaN
3   7.0  NaN  NaN
4   1.0  NaN  NaN
5   NaN  0.0  NaN
6   NaN  1.0  NaN
7   NaN  7.0  NaN
8   NaN  3.0  NaN
9   NaN  NaN  8.0
10  NaN  NaN  2.0
11  NaN  NaN  8.0


### index 有重复的情况

In [26]:
# index 有重复的情况
ser_obj1 = pd.Series(np.random.randint(0, 10, 5), index=range(5))
ser_obj2 = pd.Series(np.random.randint(0, 10, 4), index=range(4))
ser_obj3 = pd.Series(np.random.randint(0, 10, 3), index=range(3))

print(ser_obj1)
print(ser_obj2)
print(ser_obj3)

0    6
1    1
2    1
3    9
4    3
dtype: int32
0    4
1    1
2    7
3    1
dtype: int32
0    7
1    1
2    9
dtype: int32


In [27]:
#相同索引名直接往下排
print(pd.concat([ser_obj1, ser_obj2, ser_obj3]))

0    6
1    1
2    1
3    9
4    3
0    4
1    1
2    7
3    1
0    7
1    1
2    9
dtype: int32


In [30]:
#inner 和outer 代表内连接和全外连接
print(pd.concat([ser_obj1, ser_obj2, ser_obj3], axis=1, join='inner'))

   0  1  2
0  6  4  7
1  1  1  1
2  1  7  9


In [31]:
#df的concat
import pandas as pd
import numpy as np
df_obj1 = pd.DataFrame(np.random.randint(0, 10, (3, 2)), index=['a', 'b', 'c'],
                       columns=['A', 'B'])
df_obj2 = pd.DataFrame(np.random.randint(0, 10, (2, 2)), index=['a', 'b'],
                       columns=['A', 'B'])
print(df_obj1)
print(df_obj2)

print(pd.concat([df_obj1, df_obj2]))
print(pd.concat([df_obj1, df_obj2], axis=1))

   A  B
a  4  7
b  6  8
c  5  1
   A  B
a  6  8
b  0  5
   A  B
a  4  7
b  6  8
c  5  1
a  6  8
b  0  5
   A  B    A    B
a  4  7  6.0  8.0
b  6  8  0.0  5.0
c  5  1  NaN  NaN


### 数据重构

In [32]:
df_obj = pd.DataFrame(np.random.randint(0,10, (5,2)), columns=['data1', 'data2'])
print(df_obj)
#对df进行stack，就会变成mutiindex的series
stacked = df_obj.stack()  #stack内部带的参数level，选择哪一个column变为index
print(stacked)

   data1  data2
0      5      0
1      7      2
2      0      7
3      6      5
4      5      1
0  data1    5
   data2    0
1  data1    7
   data2    2
2  data1    0
   data2    7
3  data1    6
   data2    5
4  data1    5
   data2    1
dtype: int32


In [33]:
# 默认操作内层索引
print(stacked.unstack())

# 通过level指定操作索引的级别
print(stacked.unstack(level=0))

   data1  data2
0      5      0
1      7      2
2      0      7
3      6      5
4      5      1
       0  1  2  3  4
data1  5  7  0  6  5
data2  0  2  7  5  1


In [34]:
#stack和unstack对df操作
index1 = pd.MultiIndex.from_arrays([['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c', 'd', 'd', 'd'],
                [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]], names=['cloth', 'size'])

df= pd.DataFrame(np.arange(24).reshape(12,2),index=index1,columns=['data1','data2'])
print(df)
df_s=df.unstack(level=0)  #把行索引中的一个级别拿到列索引
print(df_s)
print(df_s.columns) #看到columns索引变为层级索引
df_s.stack(0)  #把columns上的0层索引放到index的内层索引

            data1  data2
cloth size              
a     0         0      1
      1         2      3
      2         4      5
b     0         6      7
      1         8      9
      2        10     11
c     0        12     13
      1        14     15
      2        16     17
d     0        18     19
      1        20     21
      2        22     23
      data1             data2            
cloth     a   b   c   d     a   b   c   d
size                                     
0         0   6  12  18     1   7  13  19
1         2   8  14  20     3   9  15  21
2         4  10  16  22     5  11  17  23
MultiIndex([('data1', 'a'),
            ('data1', 'b'),
            ('data1', 'c'),
            ('data1', 'd'),
            ('data2', 'a'),
            ('data2', 'b'),
            ('data2', 'c'),
            ('data2', 'd')],
           names=[None, 'cloth'])


cloth       a   b   c   d
size                     
0    data1  0   6  12  18
     data2  1   7  13  19
1    data1  2   8  14  20
     data2  3   9  15  21
2    data1  4  10  16  22
     data2  5  11  17  23

 ## 处理重复数据

In [35]:
df_obj = pd.DataFrame({'data1' : ['a'] * 4 + ['b'] * 4,
                       'data2' : np.random.randint(0, 4, 8)})
print(df_obj)

  data1  data2
0     a      0
1     a      1
2     a      2
3     a      2
4     b      2
5     b      2
6     b      3
7     b      0


In [36]:
print(df_obj.duplicated())
df_obj[~df_obj.duplicated()]  # 取出不重复行 ~

0    False
1    False
2    False
3     True
4    False
5     True
6    False
7    False
dtype: bool


,data1,data2
0,a,0
1,a,1
2,a,2
4,b,2
6,b,3
7,b,0


In [37]:
#按照某一列去重
print(df_obj.duplicated('data2'))

0    False
1    False
2    False
3     True
4     True
5     True
6    False
7     True
dtype: bool


In [38]:
df_obj1 = pd.DataFrame({'data1' :[np.nan] * 4,
                       'data2' :list('1235')})
df_obj1

,data1,data2
0,NaN,1
1,NaN,2
2,NaN,3
3,NaN,5


In [39]:
# pd的duplicated 认为空值和空值相等的
df_obj1.duplicated('data1')
print(df_obj1.drop_duplicates('data1'))

   data1 data2
0    NaN     1


In [40]:
print(df_obj.drop_duplicates())  #删除重复行
print(df_obj.drop_duplicates('data2'))

  data1  data2
0     a      0
1     a      1
2     a      2
4     b      2
6     b      3
7     b      0
  data1  data2
0     a      0
1     a      1
2     a      2
6     b      3


In [41]:
#如果要在原有的df上去重，需要加inplace=True
df_obj

,data1,data2
0,a,0
1,a,1
2,a,2
3,a,2
4,b,2
5,b,2
6,b,3
7,b,0


In [43]:
ser_obj = pd.Series(np.random.randint(0,10,10))  # series用map
print(ser_obj)

print(ser_obj.map(lambda x : x ** 2))

0    5
1    7
2    2
3    7
4    8
5    5
6    5
7    9
8    2
9    4
dtype: int32
0    25
1    49
2     4
3    49
4    64
5    25
6    25
7    81
8     4
9    16
dtype: int64


In [44]:
ser_obj=pd.Series(np.arange(10),index=range(3,13))
# 单个值替换单个值
print(ser_obj.replace(1, -100))

# 多个值替换一个值
print(ser_obj.replace(range(6,9), -100))

# 多个值替换多个值
print(ser_obj.replace([4, 7], [-100, -200]))

3       0
4    -100
5       2
6       3
7       4
8       5
9       6
10      7
11      8
12      9
dtype: int32
3       0
4       1
5       2
6       3
7       4
8       5
9    -100
10   -100
11   -100
12      9
dtype: int32
3       0
4       1
5       2
6       3
7    -100
8       5
9       6
10   -200
11      8
12      9
dtype: int32


In [45]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                   'B': [5, 6, 7, 8, 9],
                   'C': ['a', 'b', 'c', 'd', 'e']})

In [46]:
df

,A,B,C
0,0,5,a
1,1,6,b
2,2,7,c
3,3,8,d
4,4,9,e


In [47]:
df.replace(to_replace=r'^a', value=100, regex=True)

,A,B,C
0,0,5,100
1,1,6,b
2,2,7,c
3,3,8,d
4,4,9,e


In [48]:
df.dtypes

A     int64
B     int64
C    object
dtype: object